# Correcting field intensity: calibrating accelerometer and magnetometer

## Load and inspect data
Load pickle file and inspect contents

In [ ]:
import os
import pickle

# Import necessary pyologger utilities
from pyologger.load_data.datareader import DataReader
from pyologger.load_data.metadata import Metadata
from pyologger.plot_data.plotter import *
from pyologger.process_data.sampling import upsample
from pyologger.calibrate_data.zoc import *
from pyologger.plot_data.plotter import plot_depth_correction
from pyologger.calibrate_data.calibrate_acc_mag import *

# Change the current working directory to the root directory
# os.chdir("/Users/fbar/Documents/GitHub/pyologger")
os.chdir("/Users/jessiekb/Documents/GitHub/pyologger")

root_dir = os.getcwd()
data_dir = os.path.join(root_dir, "data")
color_mapping_path = os.path.join(root_dir, "color_mappings.json")

# Verify the current working directory
print(f"Current working directory: {root_dir}")

In [ ]:
# Initialize the info class
metadata = Metadata()
metadata.fetch_databases(verbose=False)

# Save databases
dep_db = metadata.get_metadata("dep_DB")
logger_db = metadata.get_metadata("logger_DB")
rec_db = metadata.get_metadata("rec_DB")
animal_db = metadata.get_metadata("animal_DB")

# Assuming you have the metadata and dep_db loaded:
datareader = DataReader()
deployment_folder = datareader.check_deployment_folder(dep_db, data_dir)

if deployment_folder:
    datareader.read_files(metadata, save_csv=True, save_parq=True)

In [ ]:
# Load the data_reader object from the pickle file
pkl_path = os.path.join(deployment_folder, 'outputs', 'data.pkl')

with open(pkl_path, 'rb') as file:
    data_pkl = pickle.load(file)

for logger_id, info in data_pkl.info.items():
    sampling_frequency = info.get('datetime_metadata', {}).get('fs', None)
    if sampling_frequency is not None:
        # Format the sampling frequency to 5 significant digits
        print(f"Sampling frequency for {logger_id}: {sampling_frequency} Hz")
    else:
        print(f"No sampling frequency available for {logger_id}")

## Calibrate ACC & MAG

In [ ]:
# Assuming `data_pkl` is already loaded and contains your data
accX = data_pkl.data['CC-96']['accX'].values
accY = data_pkl.data['CC-96']['accY'].values
accZ = data_pkl.data['CC-96']['accZ'].values
magX = data_pkl.data['CC-96']['magX'].values
magY = data_pkl.data['CC-96']['magY'].values
magZ = data_pkl.data['CC-96']['magZ'].values

# Combine the accelerometer and magnetometer data into nx3 matrices
acc_data = np.vstack((accX, accY, accZ)).T
mag_data = np.vstack((magX, magY, magZ)).T

# Get the sampling rate from the data structure
sampling_rate = int(data_pkl.info['CC-96']['datetime_metadata']['fs'])

# Call the check_AM function
AMcheck = compute_field_intensity_and_inclination(acc_data, mag_data, sampling_rate)

# Access the field intensity and inclination angle
field_intensity_acc = AMcheck['field_intensity'][:, 0]  # Field intensity of accelerometer data
field_intensity_mag = AMcheck['field_intensity'][:, 1]  # Field intensity of magnetometer data
inclination_angle = AMcheck['inclination_angle']

# Print results
print("Field Intensity:\n", field_intensity_acc)
print("Field Intensity:\n", field_intensity_mag)
print("Inclination Angle (degrees):\n", inclination_angle)

In [ ]:
# Save new fields to data_pkl.data['CC-96']
data_pkl.data['CC-96']['field_intensity_acc'] = field_intensity_acc
data_pkl.data['CC-96']['field_intensity_mag'] = field_intensity_mag
data_pkl.data['CC-96']['inclination_angle'] = inclination_angle

# Prepare channels to plot
imu_channels_to_plot = ['accX', 'accY', 'accZ', 'field_intensity_acc', 'field_intensity_mag', 'inclination_angle']
ephys_channels_to_plot = []
imu_logger_to_use = 'CC-96'
ephys_logger_to_use = 'UF-01'

# Get the overlapping time range
imu_df = data_pkl.data[imu_logger_to_use]
ephys_df = data_pkl.data[ephys_logger_to_use]
start_time = max(imu_df['datetime'].min(), ephys_df['datetime'].min()).to_pydatetime()
end_time = min(imu_df['datetime'].max(), ephys_df['datetime'].max()).to_pydatetime()

plot_tag_data_interactive(data_pkl, imu_channels_to_plot, ephys_channels=ephys_channels_to_plot, 
                          imu_logger=imu_logger_to_use, ephys_logger=ephys_logger_to_use, 
                          time_range=(start_time, end_time), color_mapping_path=color_mapping_path)

### Apply adjustments

In [ ]:
# Apply the fix_offset_3d function to adjust accelerometer data
result = estimate_offset_triaxial(acc_data)

# Extract the adjusted data and calibration info
adjusted_data_acc = result['X']
calibration_info_acc = result['G']

print("Adjusted Data:\n", adjusted_data_acc)
print("Calibration Info:\n", calibration_info_acc)

data_pkl.data['CC-96']['accX_adjusted'] = adjusted_data_acc[:, 0]
data_pkl.data['CC-96']['accY_adjusted'] = adjusted_data_acc[:, 1]
data_pkl.data['CC-96']['accZ_adjusted'] = adjusted_data_acc[:, 2]
data_pkl.info['CC-96']['calibration_info'] = {}
data_pkl.info['CC-96']['calibration_info'] = calibration_info_acc

# Apply the fix_offset_3d function to adjust magnetometer data
result = estimate_offset_triaxial(mag_data)

# Extract the adjusted data and calibration info
adjusted_data_mag = result['X']
calibration_info_mag = result['G']

print("Adjusted Data:\n", adjusted_data_mag)
print("Calibration Info:\n", calibration_info_mag)

data_pkl.data['CC-96']['magX_adjusted'] = adjusted_data_mag[:, 0]
data_pkl.data['CC-96']['magY_adjusted'] = adjusted_data_mag[:, 1]
data_pkl.data['CC-96']['magZ_adjusted'] = adjusted_data_mag[:, 2]
data_pkl.info['CC-96']['calibration_info'] = {}
data_pkl.info['CC-96']['calibration_info'] = calibration_info_mag

### Re-run check AM. 
Re-run the field intensity and inclination angle calculations to see if now the values are closer to expected.

In [ ]:
# Assuming `data_pkl` is already loaded and contains your data
accX = data_pkl.data['CC-96']['accX_adjusted'].values
accY = data_pkl.data['CC-96']['accY_adjusted'].values
accZ = data_pkl.data['CC-96']['accZ_adjusted'].values
magX = data_pkl.data['CC-96']['magX_adjusted'].values
magY = data_pkl.data['CC-96']['magY_adjusted'].values
magZ = data_pkl.data['CC-96']['magZ_adjusted'].values

# Combine the accelerometer and magnetometer data into nx3 matrices
acc_data = np.vstack((accX, accY, accZ)).T
mag_data = np.vstack((magX, magY, magZ)).T

# Get the sampling rate from the data structure
sampling_rate = int(data_pkl.info['CC-96']['datetime_metadata']['fs'])

# Call the check_AM function
AMcheck2 = compute_field_intensity_and_inclination(acc_data, mag_data, sampling_rate)

# Access the field intensity and inclination angle
field_intensity_acc2 = AMcheck2['field_intensity'][:, 0]  # Field intensity of accelerometer data
field_intensity_mag2 = AMcheck2['field_intensity'][:, 1]  # Field intensity of magnetometer data
inclination_angle2 = AMcheck2['inclination_angle']

# Print results
print("Field Intensity:\n", field_intensity_acc2)
print("Field Intensity:\n", field_intensity_mag2)
print("Inclination Angle (degrees):\n", inclination_angle2)

# Save new fields to data_pkl.data['CC-96']
data_pkl.data['CC-96']['field_intensity_acc2'] = field_intensity_acc2
data_pkl.data['CC-96']['field_intensity_mag2'] = field_intensity_mag2
data_pkl.data['CC-96']['inclination_angle2'] = inclination_angle2

# Example usage in Streamlit
imu_channels_to_plot = ['accX', 'accY', 'accZ', 'field_intensity_acc2', 'field_intensity_mag2', 'inclination_angle2']
ephys_channels_to_plot = []
imu_logger_to_use = 'CC-96'
ephys_logger_to_use = 'UF-01'

# Get the overlapping time range
imu_df = data_pkl.data[imu_logger_to_use]
ephys_df = data_pkl.data[ephys_logger_to_use]
start_time = max(imu_df['datetime'].min(), ephys_df['datetime'].min()).to_pydatetime()
end_time = min(imu_df['datetime'].max(), ephys_df['datetime'].max()).to_pydatetime()

plot_tag_data_interactive(data_pkl, imu_channels_to_plot, ephys_channels=ephys_channels_to_plot, 
                          imu_logger=imu_logger_to_use, ephys_logger=ephys_logger_to_use, 
                          time_range=(start_time, end_time), color_mapping_path=color_mapping_path)



### Fix inclination angle
Here is where you would fix the axes of triaxial sensor data (if the sensor axis differs from the tag axis)

### Save data

In [ ]:
# Save the new added data to the data structure
with open(pkl_path, 'wb') as file:
        pickle.dump(data_pkl, file)